In [31]:
import os
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import random
import re
load_dotenv()

True

In [32]:
api_key=os.getenv("DEEPSEEK_API_KEY")
base_url=os.getenv("DEEPSEEK_BASE_URL") 

"openai/o3-mini"
"deepseek/deepseek-r1-distill-qwen-32b"
"deepseek/deepseek-r1:free"

In [33]:
client=OpenAI(api_key=api_key, base_url=base_url)
messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
response=client.chat.completions.create(model="deepseek/deepseek-r1-distill-qwen-32b", messages=messages)
content=response.choices[0].message.content
print(content)

The meaning of life is a deeply personal and philosophical question that varies greatly among individuals, cultures, and belief systems. Some people find meaning in relationships, others in personal achievements, spirituality, or contributing to society. There's no one-size-fits-all answer, and exploring this question is a journey of self-discovery unique to each person.


In [34]:
base_path = Path("/home/aayush/rsys_data")
save_dir = base_path / "rsys_2025"
save_dir.mkdir(parents=True, exist_ok=True)  # Create the directory if it doesn't exist

feather_path = save_dir / "interactions.feather"
df = pd.read_feather(feather_path)

In [35]:
news = pd.read_csv(
    base_path/ Path("news.tsv"), 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
print(f"The article data consist in total of {len(news)} number of articles.")
news

The article data consist in total of 101527 number of articles.


,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
...,...,...,...,...,...,...,...,...
101522,N115249,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
101523,N64337,finance,finance-real-estate,Mansion Monday: Contemporary Des Moines home i...,Among the perks of this unique Des Moines home...,https://assets.msn.com/labs/mind/BBWzQq8.html,"[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W..."
101524,N100102,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]
101525,N74617,autos,autossports,Best Sports Car Deals for October,NaN,https://assets.msn.com/labs/mind/BBy5rVe.html,"[{""Label"": ""Peugeot RCZ"", ""Type"": ""V"", ""Wikida...",[]


In [ ]:

# Assuming df and news_df are your DataFrames
user_id = "U87243"  # Example user ID

# Get the row for the user
user_row = df[df["userId"] == user_id].iloc[0]

# Extract impressions and remove the '-0' or '-1' suffix
impressions = [item.split("-")[0] for item in user_row["impressions"].split()]

# Get the click history
click_history = user_row["click_history"]

# Retrieve news titles for impressions and history
news_map = dict(zip(news["itemId"], news["title"]))  # Create a mapping

# Get titles for impressions
impression_titles = [(item, news_map.get(item, "Unknown Title")) for item in impressions]

# Get titles for click history
click_titles = [(item, news_map.get(item, "Unknown Title")) for item in click_history]

# Select additional random items to reach 50 impressions
available_items = set(news["itemId"]) - set(impressions) - set(click_history)
random_items = random.sample(list(available_items), max(0, 300 - len(impressions)))
random_titles = [(item, news_map.get(item, "Unknown Title")) for item in random_items]

# Final 50 impressions
final_impressions = impression_titles + random_titles
final_impressions = final_impressions[:50]  # Ensure exactly 50

# Construct prompt
prompt = f"""
You are a slate generator. Given a user's interaction history, select the best 10 items to recommend.

User ID: {user_id}

Click History:
{', '.join([f"{item} ({title})" for item, title in click_titles])}

Candidate Impressions (50 total):
{', '.join([f"{item} ({title})" for item, title in final_impressions])}

Return the 10 most relevant items for the user from these impressions.
"""

print(prompt)

# Call OpenAI API
messages = [{"role": "user", "content": prompt}]
response = client.chat.completions.create(model="deepseek/deepseek-r1-distill-qwen-32b", messages=messages)
recommended_slate = response.choices[0].message.content

print("Recommended Slate:", recommended_slate)



You are a slate generator. Given a user's interaction history, select the best 10 items to recommend.

User ID: U87243

Click History:
N8668 (Milania Giudice Says She'll See Dad Joe 'Soon' After He Leaves ICE Custody), N39081 (What, Exactly, Is Tulsi Gabbard Up To?), N65259 (Korean Pop Star Sulli Dies at 25), N79529 (3 Foods You Should Eat at Breakfast If You Want to Lose Belly Fat, According to a Dietitian), N73408 (TobyMac says late son 'had an untamable grand personality,' shares last text exchange), N43615 (A race against time to slow Sonoma fire before monster winds return), N29379 (Costco workers reveal 33 things they'd love to tell shoppers, but can't), N32031 (What Is Mirin and Why Should You Use It?), N110232 (Renovations To Make   and Skip   Before Selling Your Home), N101921 (Scandal brings election risk for rising Democratic star), N12614 (14 of the world's most terrifying airport runways), N129591 (Heading to Disneyland? These Under-the-Radar Treats Are Must Eats), N10576

In [17]:
df

,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N94157,"[N78206, N26368, N7578, N58592, N19858, N58258..."
1,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N78699,"[N78206, N26368, N7578, N58592, N19858, N58258..."
2,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N71090,"[N78206, N26368, N7578, N58592, N19858, N58258..."
3,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,N31174,"[N78206, N26368, N7578, N58592, N19858, N58258..."
4,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,N25587,"[N47996, N82719, N117066, N8491, N123784, N214..."
...,...,...,...,...,...,...,...
1769833,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N39887,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1769834,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N110709,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1769835,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N24001,"[N39887, N22811, N110709, N1923, N24001, N7667..."
1769836,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,N127572,"[N39887, N22811, N110709, N1923, N24001, N7667..."


In [24]:
# Function to extract only valid news item IDs (e.g., "N12345")
actual_presented_slate = set(user_row["presented_slate"])
recommended_slate_set = set(recommended_slate.split(", "))
def extract_news_ids(text_set):
    news_ids = set()
    pattern = re.compile(r"N\d+")  # Matches items like "N12345"
    
    for text in text_set:
        matches = pattern.findall(text)  # Find all occurrences of "Nxxxxx"
        news_ids.update(matches)  # Add valid IDs to the set

    return news_ids

# Clean the recommended slate
clean_recommended_slate = extract_news_ids(recommended_slate_set)

# Print the cleaned slate
print("Cleaned Recommended Slate:", clean_recommended_slate)
print("Actual Presented Slate:", actual_presented_slate)

# Compute matching articles
num_matches = len(clean_recommended_slate & actual_presented_slate)  # Intersection

# Compute precision and recall
precision = num_matches / len(clean_recommended_slate) if clean_recommended_slate else 0
recall = num_matches / len(actual_presented_slate) if actual_presented_slate else 0

# Print results
print(f"Number of matching articles: {num_matches}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Cleaned Recommended Slate: {'N94157', 'N78206', 'N18478', 'N7578', 'N58258', 'N97088', 'N31174', 'N10721', 'N86972', 'N40282'}
Actual Presented Slate: {'N71090', 'N37924', 'N78206', 'N7578', 'N108809', 'N31174', 'N40282', 'N18478', 'N26368', 'N39404', 'N58592', 'N94157', 'N2591', 'N27822', 'N58258', 'N97778', 'N19858', 'N78699', 'N32954'}
Number of matching articles: 7
Precision: 0.7000
Recall: 0.3684


In [ ]:
candidate_items=300
# Function to extract valid news item IDs (e.g., "N12345")
def extract_news_ids(text_set):
    news_ids = set()
    pattern = re.compile(r"N\d+")  # Matches items like "N12345"
    
    for text in text_set:
        matches = pattern.findall(text)  # Find all occurrences of "Nxxxxx"
        news_ids.update(matches)  # Add valid IDs to the set

    return news_ids

# Sample 2000 unique user IDs
random_users = random.sample(df["userId"].unique().tolist(), 5)

# Create an empty list to store results
results = []

# Process each user
for user_id in random_users:
    # Extract user's row
    user_row = df[df["userId"] == user_id].iloc[0]
    
    # Extract actual presented slate
    actual_presented_slate = set(user_row["presented_slate"])
    
    # Extract impressions (removing "-0" or "-1" suffixes)
    user_impressions = [imp.split("-")[0] for imp in user_row["impressions"].split()]
    
    # Sample additional news items from news_df to make total 50
    all_news_items = set(news["itemId"]) - set(user_impressions)  # Avoid duplicates
    additional_news = random.sample(list(all_news_items), max(0, candidate_items - len(user_impressions)))  # Fill up to 50
    
    # Final expanded impressions
    final_impressions = list(set(user_impressions) | set(additional_news))
    
    # Extract titles for impressions
    impression_titles = {
        row["itemId"]: row["title"] for _, row in news[news["itemId"].isin(final_impressions)].iterrows()
    }
    
    # Extract click history (removing "-0"/"-1")
    click_history_items = user_row["click_history"]
    
    # Extract titles for click history
    click_history_titles = {
        row["itemId"]: row["title"] for _, row in news[news["itemId"].isin(click_history_items)].iterrows()
    }
    
    # Format click history with titles
    formatted_click_history = "\n".join(
        [f"{item}: {click_history_titles.get(item, 'Unknown Title')}" for item in click_history_items]
    )
    
    # Format impressions with titles
    formatted_impressions = "\n".join(
        [f"{item}: {impression_titles.get(item, 'Unknown Title')}" for item in final_impressions]
    )
    
    # Create a formatted prompt
    prompt_content = (
        f"User ID: {user_id}\n"
        f"Click History:\n{formatted_click_history}\n\n"
        f"Candidate items:\n{formatted_impressions}\n\n"
        f"Recommend the best 10 news items for this user from the set of candidate items with the Itemids from the impressions."
    )
    
    # Generate recommended slate from OpenAI
    messages = [{"role": "user", "content": prompt_content}]
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1-distill-qwen-32b",
        messages=messages
    )
    openai_response = response.choices[0].message.content
    # Extract recommended slate from OpenAI response
    recommended_slate_raw = response.choices[0].message.content.split("\n")
    recommended_slate_set = extract_news_ids(recommended_slate_raw)  # Clean and extract IDs
    
    # Compute matching articles
    num_matches = len(recommended_slate_set & actual_presented_slate)
    
    # Compute precision and recall
    precision = num_matches / len(recommended_slate_set) if recommended_slate_set else 0
    recall = num_matches / len(actual_presented_slate) if actual_presented_slate else 0

    # Append to results list
    results.append({
        "userId": user_id,
        "recommended_slate": list(recommended_slate_set),
        "presented_slate": list(actual_presented_slate),
        "number_of_matches": num_matches,
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "openai_response": openai_response 
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Display first few rows
print(results_df.head())

# Optional: Save to CSV
# results_df.to_csv("recommendation_metrics.csv", index=False)


    userId                                  recommended_slate  \
0  U509893  [N56937, N76665, N117365, N34172, N73338, N880...   
1  U107349  [N17575, N18508, N73137, N108809, N130265, N11...   
2  U453214  [N33658, N122489, N62180, N66289, N49816, N189...   
3  U129646  [N40625, N32051, N75646, N1767, N241, N14675, ...   
4  U495925  [N100856, N44122, N70822, N85806, N3094, N1295...   

                                     presented_slate  number_of_matches  \
0  [N114449, N56873, N34172, N50175, N128412, N13...                  7   
1  [N123362, N17575, N73137, N32154, N83421, N130...                  7   
2  [N18947, N110303, N7937, N38902, N119838, N790...                  1   
3  [N22627, N74401, N46716, N64785, N75646, N4578...                 10   
4  [N98178, N70822, N93643, N49168, N55792, N1168...                  3   

   precision  recall                                    openai_response  
0        0.7  0.1707  Based on the user's click history, which shows...  
1        0

In [30]:
results_df.iloc[1]["openai_response"]


"**Recommended News Items for User U107349:**\n\nBased on the user's click history showing interests in sports, celebrity news, automotive topics, and real estate, the following 10 items are recommended:\n\n1. **N17575: Big celebrity money moments of the last decade**  \n   - Aligns with celebrity interest and financial aspects.\n\n2. **N74360: Kevin Hart Makes First Official Appearance at People's Choice Awards 2019 Following Near-Fatal Accident**  \n   - Celebrities and current events.\n\n3. **N108809: Week in celebrity photos for Nov. 4-8, 2019**  \n   - Celebrity visuals and gossip.\n\n4. **N130265: Labor unrest again lurks just below the surface as baseball's Hot Stove heats up**  \n   - Baseball labor news, aligning with sports interest.\n\n5. **N118600: Miami Heat sign Chris Silva to two-way contract**  \n   - NBA news, fitting sports focus.\n\n6. **N18508: Top takeaways from Sunday's Week 10 NFL action**  \n   - NFL summaries, relevant to sports fans.\n\n7. **N80513: These Are 